# Distribuez votre projet

* Une distribution est une collection de modules Python distribués ensembles comme une ressource téléchargeable et destinée à être installée. Il existe énormément de distributions de modules comme *PIL (the Python Imaging Library)*, *PyXML*, *etc*.

* Pour faire le lien entre la distribution et la plateforme de destination, on utilise des outils appelés *packagers*. Les packagers vont prendre les sources et les compiler pour effectuer une «release». Ainsi les utilisateurs finaux vont pouvoir installer les modules sans difficulté.

* *Distutils* est un packager basique. Pour distribuer votre code, il faudra écrire un script d’installation (nommé `setup.py` par convention) et éventuellement écrire un fichier de configuration d’installation. Ensuite, il vous faudra créer une ressource distribuable (souvent une archive) et optionnellement créer une ou plusieurs distributions compilées.


## Ecrire le fichier setup.py

* Le script setup est généralement assez simple. Voici un premier exemple :

```python
from distutils.core import setup
setup(name='foo',
      version='1.0',
      py_modules=['foo'])
```

* La plupart des informations sont fournies comme arguments à la fonction `setup`.
* Ces arguments peuvent être regroupés en deux catégories : les metadonnées du package (nom, version) et les informations sur ce qu’est, ce que fait le package. 
* Les modules sont spécifiés par leur nom d’objet et non leur nom de fichier. Il est recommandé de fournir des metadonnées supplémentaires comme son nom, son adresse mail et une url du projet :
    * Vous pouvez lister des modules individuellement : `py_modules = ['mod1', 'pkg.mod2']`
    * ou lister des packages entiers : `packages = ['distutils', 'distutils.command']`. Ici on spécifie des modules purs Python par package plutôt que de lister tous les modules de ce paquet.
    * Par exemple, ce package :

```python
    setup.py
    src/
        mypkg/
            __init__.py
            module.py
            data/
                tables.dat
                spoons.dat
                forks.dat
```

* pourrait avoir un setup comme cela :

```python
    setup(...,
          packages=['mypkg'],
          package_dir={'mypkg': 'src/mypkg'},
          package_data={'mypkg': ['data/*.dat']})
```        

* Voici une liste non exhaustive d’arguments de la fonction **setup**
    - le nom du projet : nom="sample"
    - la version : version="1.2.0" (voir [PEP 440](https://www.python.org/dev/peps/pep-0440/))
    - les packages à inclure dans le projet. On peut les lister ou utiliser `find_packages` pour automatiser cette tâche (`exclude` pour pour exlure ceux qui ne doivent pas être installés) : `py_modules=['foo']`
    - Metadonnées: Il est important d’inclure des métadonnées à propos de votre projet. 

        ```python

            # A description of your project
            description='A sample Python project',
            long_description=long_description,

            # The project's main homepage
            url='https://github.com/pypa/sampleproject',

            # Author details
            author='The Python Packaging Authority',
            author_email='pypa-dev@googlegroups.com',

            # Choose your license
            license='MIT',

            # See https://pypi.python.org/pypi?%3Aaction=list_classifiers
            classifiers=[
                # How mature is this project? Common values are
                #   3 - Alpha
                #   4 - Beta
                #   5 - Production/Stable
                'Development Status :: 3 - Alpha',

                # Indicate who your project is intended for
                'Intended Audience :: Developers',
                'Topic :: Software Development :: Build Tools',

                # Pick your license as you wish (should match "license" above)
                'License :: OSI Approved :: MIT License',

                # Specify the Python versions you support here. In particular, ensure
                # that you indicate whether you support Python 2, Python 3 or both.
                'Programming Language :: Python :: 2',
                'Programming Language :: Python :: 2.6',
                'Programming Language :: Python :: 2.7',
                'Programming Language :: Python :: 3',
                'Programming Language :: Python :: 3.2',
                'Programming Language :: Python :: 3.3',
                'Programming Language :: Python :: 3.4',
            ],

            # What does your project relate to?
            keywords='sample setuptools development',
        ```

- Enfin, il est possible d'ajouter :
    - Les dépendances : install_requires = ['peppercorn']
        "install_requires" est utilisé pour spécifier quelles dépendances sont nécessaire au projet pour fonctionner. Ces dépendances seront installés par Pip lors de l'installation de votre projet.
    - Fichiers additionnels : package_data = { 'sample': ['package_data.dat']}
- Voici un exemple plus complet :

    ```python
    #! /usr/bin/env python3

    from distutils.core import setup

    setup(name='Distutils',
          version='1.0',
          description='Python Distribution Utilities',
          author='Greg Ward',
          author_email='gward@python.net',
          url='https://www.python.org/sigs/distutils-sig/',
          packages=['distutils', 'distutils.command'],
         )

    ```

* Vous pouvez également inclure d’autre fichier au package comme un ***README*** pour expliquer le projet et un ***MANIFEST*** pour définir des fichiers supplémentaires à inclure dans la distribution du projet packagé. 



## Utiliser le fichier setup.py

Nous allons voir qu'un même ```setup.py``` est utilisé aussi bien pour créer une distribution que pour installer la distribution.  

- Pour créer une distribution des fichiers sources du module, il faut donc créer un script d’installation (setup.py) contenant le code ci-dessus et exécuter la commande 
```
$>python setup.py sdist [$>sdist setup.py sous windows]
```

- sdist va créer une archive (tarball sous unix et zip sous windows) contenant le script setup.py et le module. Le fichier d’archive sera nommé foo-1.0.tar.gz (ou .zip) et sera décompressé dans un répertoire foo-1.0.



- Pour installer le module, après avoir télécharger et décompresser l’archive, il faut se déplacer dans le répertoire créé par la décompression de l’archive et taper la commande suivante :

```
$>python setup.py install
```

- Cette commande va copier les fichiers dans le répertoire réservé aux modules tiers de l’installation Python. 
- Remarque : On note donc que c’est le même script qui sert pour la distribution et l’installation.


- On peut faciliter encore plus l’installation des modules distribués. Par exemple, sous windows, on peut créer un installateur exécutable avec cette commande :

```
$>python setup.py bdist_wininst
```

- Cette commande va créer un `.exe` nommé foo-1.0.win32.exe dans le répertoire courant.
- Il existe également d’autre format de distribution : le rpm avec bdist_rpm, le pkgtool (bdist_pkgtool) et le hp_ux swinstall (bdist_sdux)
- Vous pouvez lister les formats de distribution disponibles avec cette commande :

```
$>python setup.py bdist –help-formats
```



## Le mode développement

Setuptools (l'extension de Distutils) offre un mode "développement", qui permet de faire semblant d'installer un package, mais de n'installer en fait que des liens symboliques qui pointent vers les fichiers originaux, de telle sorte qu'on peut éditer ces fichiers et tester leur utilisation par du code client, sans avoir à les réinstaller à chaque fois. De plus, une commande permet de nettoyer la fausse installation si nécessaire :
* pour pseudo-installer une distribution : `python setup.py develop`
* pour nettoyer : `python setup.py develop --uninstall`

## Publier ses packages

Pour installer des packages, on vous conseille d’utiliser `pip` (Python installing package). Cet outil cherche les packages sur le Python Package Index (PyPi). Les packages Python peuvent être compactés dans des archives tarball ou zip. Python utilise des formats de distribution. Actuellement, Python utilise egg mais ce format va etre peu à peu remplacé par wheel. Pour construire votre package, vous pouvez donc utiliser **[wheel](http://pythonwheels.com/)** et pour envoyer votre package sur Pypi, il faut utiliser l’outil **[twine](https://github.com/pypa/twine)**

## A propos des auteurs

*Travail initié en 2014 dans le cadre d'une série de formations Python organisées par le réseau Devlog.  
Auteur principal : . 
Contributions de : Christophe Gengembre.
Relecteurs : Nicolas Can, Sekou Diakite, Christophe Halgand, Loic Gouarin, David Chamont.*

### Mise en forme

In [ ]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()